In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.models import PrositIntensityPredictor
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
print([x for x in dir(dlomix) if not x.startswith("_")])

2023-07-04 13:35:18.934773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 13:35:19.088096: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-04 13:35:19.860451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-04 13:35:19.860518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

['META_DATA', 'constants', 'data', 'eval', 'layers', 'losses', 'models', 'pipelines', 'reports', 'utils']


In [3]:
from dlomix.constants import ALPHABET_UNMOD

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [6]:
project_name = "transforming-prosit"

In [6]:
df_annotation = pd.read_parquet('/cmnfs/proj/prosit/Transformer/TUM_first_pool_1/TUM_first_pool_59_01_01_annotation.parquet')

In [7]:
df_annotation.head()

,ion_type,no,charge,experimental_mass,theoretical_mass,intensity,neutral_loss,fragment_score,peptide_sequence,scan_number,raw_file
0,b,6,2,266.15887,266.105904,0.03,C3H5NO-C3H6,39,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
1,b,6,2,284.12894,283.632089,0.05,C2H4O2-H2O,64,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
2,b,6,2,284.12894,284.124097,0.05,C2H4O2-NH3,64,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
3,b,6,2,284.12894,284.124097,0.05,CH3NO-CH4O,39,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2
4,b,6,2,284.12894,284.124097,0.05,C2H5NO-H2O,64,ESQVAEEEEWLHK,27664,01650b_BA3-TUM_first_pool_59_01_01-DDA-1h-R2


In [9]:
df = pd.read_parquet('/cmnfs/proj/prosit/Transformer/Final_Meta_Data/TUM_first_pool_meta_data.parquet')


In [10]:
df.head()

,raw_file,scan_number,modified_sequence,precursor_charge,precursor_intensity,mz,precursor_mz,fragmentation,mass_analyzer,retention_time,indexed_retention_time,andromeda_score,peptide_length,orig_collision_energy,aligned_collision_energy
0,01625b_GA1-TUM_first_pool_1_01_01-3xHCD-1h-R1,9465,KLVEQLR,2,997172400.0,443.27945,443.278897,HCD,FTMS,19.537,16.363542,277.39,7,25.0,27.352073
1,01625b_GA1-TUM_first_pool_1_01_01-3xHCD-1h-R1,16689,QRDPSSFLR,2,64341670.0,553.29108,553.290600,HCD,FTMS,25.679,32.344600,275.01,9,35.0,37.352073
2,01625b_GA1-TUM_first_pool_1_01_01-3xHCD-1h-R1,23310,QGEQIEIIR,2,357365700.0,543.30111,543.300696,HCD,FTMS,31.364,47.136575,272.82,9,25.0,27.352073
3,01625b_GA1-TUM_first_pool_1_01_01-3xHCD-1h-R1,26115,QELTQLC[UNIMOD:4]LK,2,194229200.0,566.80517,566.804896,HCD,FTMS,33.763,53.378607,266.03,9,25.0,27.352073
4,01625b_GA1-TUM_first_pool_1_01_01-3xHCD-1h-R1,23409,QGEQIEIIR,2,458835200.0,543.30111,543.300696,HCD,FTMS,31.449,47.357739,265.58,9,30.0,32.352073


In [13]:
df["fragmentation"].unique()

array(['HCD', 'CID'], dtype=object)